In [1]:
# usual imports
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np

In [2]:
# local paths
msd_subset_path= '/Users/dannynightingale/Downloads/MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check

In [3]:
# imports specific to the MSD
import hdf5_getters as GETTERS

In [4]:
# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

# we define this very useful function to iterate the files
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [5]:
# we can now easily count the number of files in the dataset
print('number of song files:',apply_to_all_files(msd_subset_data_path))

number of song files: 10000


In [21]:
all_artist_names = set()

# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()
    
# let's apply the previous function to all files
# we'll also measure how long it takes
t1 = time.time()
apply_to_all_files(msd_subset_data_path, func=func_to_get_artist_name)
t2 = time.time()
print('all artist names extracted in:', strtimedelta(t1,t2))

all artist names extracted in: 0:01:49.504729


In [23]:
list(all_artist_names)[:4]

[b'Embrace the End',
 b'Edwin Hawkins',
 b'Wyclef Jean featuring Butch Cassidy and Sharissa',
 b'Harry Diboula']

In [16]:
all_artist_names = []
all_title = []

# we define the function to apply to all files
def func_to_get_data(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    title = GETTERS.get_title(h5)
    hotness = GETTERS.get_song_hotttnesss(h5)
    hotness = GETTERS.get_song_hotttnesss(h5)
    
    all_artist_names.append(GETTERS.get_artist_name(h5))
    all_title.append(GETTERS.get_title(h5))
    
    h5.close()

t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_data)
t2 = time.time()
print('all artist names extracted in:',strtimedelta(t1,t2))    
print(len(all_artist_names))
print(len(all_title))

all artist names extracted in: 0:01:59.905603
10000
10000


In [25]:
print(all_title[320])
print(list(all_artist_names)[320])

[b'Stille nacht_ heilige nacht', b'Nine Ways', b'Northern Downpour (Album Version)', b'Bad Seed', b'Exchange', b"Marie D'Aquitaine", b'Oh My God', b'Joy To The World', b'Hoochie Coochie Man', b'Ftei O Erotas', b'The Second Summer Of Love', b'Prelude And Fugue In C Major BWV 531 : Fugue', b"Don't Phunk With My Heart", b'Give It What U Got', b'Dancing To A Standstill', b'Isla del Luna', b'Spirit', b'Another Long One', b'Leng Jing', b'OAKtown']
[b'Bruno Sanfilippo', b'Blackbyrds', b'Dolores Keane', b'A-Studio', b'Deep Puddle Dynamics', b'The Stolen Minks', b'Rance Allen Group', b'Charanga Forever', b'Francis Dunnery', b'Pascal Obispo', b'Alturas', b'Jimmy Wakely', b'King Curtis with Jack Dupree', b'The Wreckers', b'Bill Perry', b'Arthur Brown', b'Peter Spence', b'X-ecutioners featuring M.O.P.', b'Ras Michael and the Sons of Negus', b'Yvette Horner']


In [20]:
# let's see some of the content of 'all_artist_names'
print('found',len(all_artist_names),'unique artist names')
for k in range(5):
    print(list(all_artist_names)[k])

# this is too long, and the work of listing artist names has already
# been done. Let's redo the same task using an SQLite database.
# We connect to the provided database: track_metadata.db
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
# we build the SQL query
q = "SELECT DISTINCT artist_name FROM songs"
# we query the database
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print('all artist names extracted (SQLite) in:',strtimedelta(t1,t2))
# we close the connection to the database
conn.close()
# let's see some of the content
for k in range(5):
    print(all_artist_names_sqlite[k][0])
    
print(len(all_artist_names_sqlite))

found 10000 unique artist names
b'Casual'
b'The Box Tops'
b'Sonora Santanera'
b'Adam Ant'
b'Gob'
all artist names extracted (SQLite) in: 0:00:00.150146
!!!
(hed) p.e.
089 Clique feat. Minnesota Snipe & Skinny Cueball
089 Clique feat. Prophet
1. Futurologischer Congress
4412


In [13]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))

# we build the SQL query
q = "SELECT * FROM songs"
# we query the database
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print('all artist names extracted (SQLite) in:', strtimedelta(t1,t2))
# we close the connection to the database
conn.close()
# let's see some of the content
for k in range(30):
    print(all_artist_names_sqlite[k])
    
print(len(all_artist_names_sqlite))

all artist names extracted (SQLite) in: 0:00:00.062497
('TRACCVZ128F4291A8A', 'Deep Sea Creature', 'SOVLGJY12A8C13FBED', 'Call of the Mastodon', 'ARMQHX71187B9890D3', 'bc5e2ad6-0a4a-4d90-b911-e9a7e6861727', 'Mastodon', 280.21506, 0.780461748777, 0.574274730517, 2001)
('TRACCMH128F428E4CD', 'No Quieras Marcharte', 'SOGDQZK12A8C13F37C', 'Adelante', 'AR2PT4M1187FB55B1A', 'd54ea4a6-0b9c-4e47-bed0-289ae9ff4037', 'Los Chichos', 191.68608, 0.561589945857, 0.420570307208, 1984)
('TRACCSW128F148C7C3', 'If I...', 'SODMVJR12A6D4F985D', 'Ill Na Na', 'ARDI88R1187B98DAB2', 'fd87374e-ffde-4d36-89a8-8a073f795666', 'Foxy Brown', 222.92853, 0.687687485872, 0.406686099467, 0)
('TRACCXJ128F428F0CF', "Werther - Lyric Drama in four Acts/Act I/Alors_ c'est bien ici la maison du bailli?", 'SOIWBDR12A8C13A4AC', 'Massenet: Werther', 'ARUKJUP12086C14589', '8a5f2736-bcde-4a2e-8d50-72631d66a7ef', 'Ramón Vargas;Vladimir Jurowski', 278.38649, 0.391741394148, 0.291264811753, 0)
('TRACCVS12903D014F8', 'Ad Te Clamamus 

In [ ]:
all_artist_names = set()

# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()

In [ ]:
"""
Tutorial for the Million Song Dataset

by Thierry Bertin-Mahieux (2011) Columbia University
   tb2332@columbia.edu
   Copyright 2011 T. Bertin-Mahieux, All Rights Reserved

This tutorial will walk you through a quick experiment
using the Million Song Dataset (MSD). We will actually be working
on the 10K songs subset for speed issues, but the code should
transpose seamlessly.

In this tutorial, we do simple metadata analysis. We look at
which artist has the most songs by iterating over the whole
dataset and using an SQLite database.

You need to have the MSD code downloaded from GITHUB.
See the MSD website for details:
http://labrosa.ee.columbia.edu/millionsong/

If you have any questions regarding the dataset or this tutorial,
please first take a look at the website. Send us an email
if you haven't found the answer.

Note: this tutorial is developed using Python 2.6
      on an Ubuntu machine. PDF created using 'pyreport'.
"""


# path to the Million Song Dataset code
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_code_path='/home/thierry/Columbia/MSongsDB'
assert os.path.isdir(msd_code_path),'wrong path' # sanity check
# we add some paths to python so we can import MSD code
# Ubuntu: you can change the environment variable PYTHONPATH
# in your .bashrc file so you do not have to type these lines
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

# imports specific to the MSD
import hdf5_getters as GETTERS

# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

# we define this very useful function to iterate the files
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

# we can now easily count the number of files in the dataset
print 'number of song files:',apply_to_all_files(msd_subset_data_path)

# let's now get all artist names in a set(). One nice property:
# if we enter many times the same artist, only one will be kept.
all_artist_names = set()

# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()
    
# let's apply the previous function to all files
# we'll also measure how long it takes
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_artist_name)
t2 = time.time()
print 'all artist names extracted in:',strtimedelta(t1,t2)




# let's see some of the content of 'all_artist_names'
print 'found',len(all_artist_names),'unique artist names'
for k in range(5):
    print list(all_artist_names)[k]

# this is too long, and the work of listing artist names has already
# been done. Let's redo the same task using an SQLite database.
# We connect to the provided database: track_metadata.db
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
# we build the SQL query
q = "SELECT DISTINCT artist_name FROM songs"
# we query the database
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print 'all artist names extracted (SQLite) in:',strtimedelta(t1,t2)
# we close the connection to the database
conn.close()
# let's see some of the content
for k in range(5):
    print all_artist_names_sqlite[k][0]

# now, let's find the artist that has the most songs in the dataset
# what we want to work with is artist ID, not artist names. Some artists
# have many names, usually because the song is "featuring someone else"
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id FROM songs"
res = conn.execute(q)
all_artist_ids = map(lambda x: x[0], res.fetchall())
conn.close()

# The Echo Nest artist id look like:
for k in range(4):
    print all_artist_ids[k]

# let's count the songs from each of these artists.
# We will do it first by iterating over the dataset.
# we prepare a dictionary to count files
files_per_artist = {}
for aid in all_artist_ids:
    files_per_artist[aid] = 0

# we prepare the function to check artist id in each file
def func_to_count_artist_id(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_id = GETTERS.get_artist_id(h5)
    files_per_artist[artist_id] += 1
    h5.close()

# we apply this function to all files
apply_to_all_files(msd_subset_data_path,func=func_to_count_artist_id)

# the most popular artist (with the most songs) is:
most_pop_aid = sorted(files_per_artist,
                      key=files_per_artist.__getitem__,
                      reverse=True)[0]
print most_pop_aid,'has',files_per_artist[most_pop_aid],'songs.'

# of course, it is more fun to have the name(s) of this artist
# let's get it using SQLite
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_name FROM songs"
q += " WHERE artist_id='"+most_pop_aid+"'"
res = conn.execute(q)
pop_artist_names = map(lambda x: x[0], res.fetchall())
conn.close()
print 'SQL query:',q
print 'name(s) of the most popular artist:',pop_artist_names

# let's redo all this work in SQLite in a few seconds
t1 = time.time()
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id,artist_name,Count(track_id) FROM songs"
q += " GROUP BY artist_id"
res = conn.execute(q)
pop_artists = res.fetchall()
conn.close()
t2 = time.time()
print 'found most popular artist in',strtimedelta(t1,t2)
print sorted(pop_artists,key=lambda x:x[2],reverse=True)[0]